# Реализация вычисления критериев оптимальности выбора стратегии на Python 3

## Цель исследования (работы)

## Постановка задачи

В основе лежит класс - таблица со значениями выигрыша при выборе той или иной стратегии для каждого игрока

Вид структуры данных для размерности 3x3 - 3 игрока, 3 стратегии

{'m': {'f': None, 't': None, 'a': None}, 'w': {'f': None, 't': None, 'a': None}, 'c': {'f': None, 't': None, 'a': None}}

В данном примере игроки - мужчина (m), женщина (w), ребенок (c), стратегии - "пойти на футбол" (f), "пойти в театр" (t), "пойти на аттракционы" (a).

Программа работает для любого числа игроков и любого числа стратегий

Задача: определить оптимальную стратегию для максимизации выигрыша с помощью различных критериев оптимальности

## Теоретические сведения

Рассмотренные критерии:
    1. **Критерий Гурвица**
        Критерий Гурвица рекомендует стратегию, определяемую по формуле max (A \* max i + (1-A) \* min i), где А — степень оптимизма и изменяется в пределах от 0 до 1. Критерий выдает результат, учитывающий возможность как наихудшего, так и наилучшего поведения природы;
    2. **Критерий Байеса**
        Критерий Байеса — правило, в соответствии с которым стратегия решений выбирается таким образом, чтобы обеспечить минимум среднего риска. При этом критерии учитываются вероятности выбора стратегий другими игроками с точки зрения текущего игрока. Т.е. что текущий игрок думает о стратегиях других игроков;
    3. **Критерий Вальда**
        Критерий гарантированного результата (максиминный критерий Вальда) — это пессимистический по своей сути критерий, потому что принимается во внимание только самый плохой из всех возможных результатов каждой альтернативы. Этот подход устанавливает гарантированный минимум, хотя фактический результат может и не быть настолько плохим;
    4. **Критерий Парето**
        Оптимальность по Парето — такое состояние некоторой системы, при котором значение каждого частного показателя, характеризующего систему, не может быть улучшено без ухудшения других;
    5. **Критерий Сэвиджа**
        Критерий минимаксного риска Сэвиджа можно рассматривать как критерий наименьшего вреда, который определяет худ­шие возможные последствия для каждой альтернативы и выбирает альтернативу с лучшим из плохих значений;
    6. **Критерий Лапласа**
        Значение критерия Лапласа — это наибольшее значение математического ожидания выигрыша в условиях неопределённости состояний природы, то есть наибольшее среднее арифметическое значение выигрыша;
    7. **Критерий Нэша (равновесие Нэша)**
        Равновесие Нэша — концепция решения, набор стратегий в игре для двух и более игроков, в котором ни один участник не может увеличить выигрыш, изменив свою стратегию, если другие участники своих стратегий не меняют. Точек равновесия по Нэшу может быть несколько, хотя не во всех выигрыш будет максимальный;
    8. **Оптимистический критерий**
        Критерий оптимизма (критерий максимакса) соответствует оптимистической наступательной стратегии; здесь не принимается во внимание никакой возможный результат, кроме лучшего.

О критериях можно почитать тут:

1. [Критерии Вальда, Гурвица, Сэвиджа, Оптимистический](https://lubbook.org/book_411_glava_7_2.3.KRITERII_OPTIMALNOSTI_V_.html)
2. [Критерий Нэша](https://ru.wikipedia.org/wiki/Равновесие_Нэша)
3. [Критерий Парето](https://ru.wikipedia.org/wiki/Эффективность_по_Парето)
4. [Критерий Байеса](http://scask.ru/n_book_mr.php?id=9)
5. [Критерий Лапласа](http://cyclowiki.org/wiki/Критерий_Лапласа)

## Демонстрация проекта

### Реализация класса

In [ ]:
import itertools
from numpy import prod
from numpy import linspace
from itertools import groupby
from functools import reduce

# node scheme
#{'m': {'f': None, 't': None, 'a': None}, 'w': {'f': None, 't': None, 'a': None}, 'c': {'f': None, 't': None, 'a': None}}


class Table():
    
    # инициализация игроков и стратегий
    def __init__(self, players, strats):
        self.players = players
        self.stratsInitial = strats
        self.table = []

    # установка значений выигрыша каждого из игроков в случае выбора ими определенных стратегий
    # например, для трех игроков - m, w, c в случае выбора ими стратегий f, a, a соответственно
    # значения выигрышей будут 2 (m), 5 (w), 7 (w). Т.е., женщина с ребенком, пойдя вдвоем на аттракционы
    # выиграют 5 и 7. Мужчина, пойдя один на футбол, выиграет 2.
    def set(self, path, value): # path - {'m': 'f','w': 'f','c': 'f'}; value - (1,2,3)
        elem = self.get(path)
        if not elem:
            elem = {player: {strat: None for strat in self.stratsInitial[player]} for player in self.players}
            self.table.append(elem)
        for num, player in enumerate(path):
            elem[player][path[player]] = value[num]

    # устанавливает выигрышы для всех сразу
    def setAllAtOnce(self, data):
        values = iter(data)
        stratsProduct = [p for p in itertools.product(*list(self.stratsInitial.values()))]

        for strat in stratsProduct:
            self.set({p: s for p,s in zip(self.players, strat)}, next(values))
    
    # Реализация критерия Гурвица
    def Gurwic(self, ld):
        return self.fancyPrint({player: self.GurwicLambda(self.allMinMaxStrats('min')[player], self.allMinMaxStrats('max')[player], ld) for player in self.players})
    
    
    # Реализация критерия Байеса
    def Bayes(self, opinions): # opinions - {m: {w: {f: None, t: None, a: None}, c: {f: None, t: None, a: None}}, w: {m: {f: None, t: None, a: None}, c: {f: None, t: None, a: None}}, ...}
        temp = {player: {strat: None for strat in self.stratsInitial[player]} for player in self.players}
        for player in self.players:
            for strat in self.stratsInitial[player]:
                playerStratValues = [elem[player][strat] for elem in self.modFilter(player, strat, self.table)] # список значений стратегии strat для игрока player
                stratsProduct = list(itertools.product(*[self.stratsInitial[p] for p in self.stratsInitial if p != player]))


                temp[player][strat] = sum([playerStratValues[num] * prod(self.allProbability(opinions[player], product)) for num, product in enumerate(stratsProduct)])
                # prod (перемножение), так как считаем что принятие игроками стратегий происходит независимо
        return self.fancyPrint({player: self.modMaxSearch(temp[player]) for player in self.players})
    
    # Реализация критерия Вальда
    def Valdo(self):
        return self.fancyPrint({player: self.modMaxSearch(self.allMinMaxStrats('min')[player]) for player in self.players})
    
    # Реализация критерия Парето
    def Pareto(self):
        Res = []
        for numi, i in enumerate(self.table):
            for numj, j in enumerate(self.table):
                if numi == numj: continue
                if not self.compareSituationsGains(self.getOnlyGain(i), self.getOnlyGain(j)): break
            else: Res.append(i)

        return self.fancyPrint([self.fancyCell(cell) for cell in Res])
    
    # Реализация критерия Сэвиджа
    def Sevij(self):
        allMaxStrats = self.allMinMaxStrats('max')

        maxValueStrat = {player: self.modMaxSearch(allMaxStrats[player]) for player in self.players} # {'m': {'f': 8}, 'w': {'t': 8}}

        for player in self.players:
            strategy = list(maxValueStrat[player].keys())[0]
            for strat in self.stratsInitial[player]:
                allMaxStrats[player][strat] = maxValueStrat[player][strategy] - allMaxStrats[player][strat]

        return self.fancyPrint({player: self.modMaxSearch(allMaxStrats[player]) for player in self.players})
    
    # Реализация критерий Лапласа
    def Laplas(self):
        temp = {player: {strat: None for strat in self.stratsInitial[player]} for player in self.players}
        for player in self.players:
            for strat in self.stratsInitial[player]:
                stratValues = [elem[player][strat] for elem in self.modFilter(player, strat, self.table)]
                value = sum(stratValues) / len(stratValues)
                temp[player][strat] = value

        return self.fancyPrint({player: self.modMaxSearch(temp[player]) for player in self.players})
    
    # Реализация критерия Нэша
    def Nesh(self):
        Res = []
        for cell in self.table:
            for num, player in enumerate(self.players):
                if self.getOnlyGain(cell)[num] < max([self.getNotNoneStrat(cell[player]) for cell in self.getFloatPlayerStrategySituations(player, cell)]):
                    break
            else: Res.append(cell)

        return self.fancyPrint([self.fancyCell(cell) for cell in Res])
    
    # Реализация оптимистического критерия
    def optimistic(self):
        return self.fancyPrint({player: self.modMaxSearch(self.allMinMaxStrats('max')[player]) for player in self.players})
    
    # Далее идут вспомогательные методы
    
    def get(self, path): # path - {'m': 'f','w': 'f','c': 'f'}
        res = self.table
        for player in path:
            res = list(filter(lambda cell: cell[player][path[player]] != None, res))
        if res: res = res[0]
        return res or None
    
    def modMaxSearch(self, d):
        maxS, maxV = '', -100
        for strat in d:
            if maxV < d[strat]:
                maxV, maxS = d[strat], strat
        return {maxS: maxV}

    def modFilter(self, p, s, table): # Возвращает все элементы, в которых у игрока player стратегия не None
        return list(filter(lambda elem: elem[p][s] != None, table))

    def GurwicLambda(self, minStrat, maxStrat, ld):
        maxS, maxV = '', -100
        for strat in minStrat.keys():
            temp = minStrat[strat] * (1 - ld) + maxStrat[strat] * ld
            if maxV < temp:
                maxV, maxS = temp, strat
        return {maxS: maxV}

    def allMinMaxStrats(self, mod): # Возвращает макс. или мин. значения всех стратегий для всех игроков
        allStratsMinMax = {player: {strat: None for strat in self.stratsInitial[player]} for player in self.players}

        for player in self.players:
            for strat in self.stratsInitial[player]:
                if mod == 'max':
                    allStratsMinMax[player][strat] = max(
                        list(map(lambda elem: elem[player][strat], self.modFilter(player, strat, self.table))))
                elif mod == 'min':
                    allStratsMinMax[player][strat] = min(
                        list(map(lambda elem: elem[player][strat], self.modFilter(player, strat, self.table))))
        return allStratsMinMax

    def compareSituationsGains(self, firstGains, secondGains):
        compareRes = list(map(lambda couple: couple[0] > couple[1], zip(firstGains, secondGains)))
        if compareRes.count(True) != 0: return True
        else: return False

    def getOnlyGain(self, cell): # возвращает кортеж выигрышей игроков для конкретной ситуации cell
        return [tuple(filter(lambda strat: strat != None, list(cell[player].values())))[0] for player in cell.keys()]

    def fancyCell(self, cell):
        fancy = {}
        for player in self.players:
            for strat in self.stratsInitial[player]:
                if cell[player][strat] != None:
                    fancy.update({player: {strat: cell[player][strat]}})
        return fancy

    def getFloatPlayerStrategySituations(self, player, cell): # возвращает все ситуации, в которых стратегия player не фиксирована, а стратегии остальных как в cell
        res = []
        for p in cell:
            if p == player: continue
            for strat in cell[p]:
                if cell[p][strat] != None:
                    res = self.modFilter(p, strat, res or self.table)
        return res

    def getNotNoneStrat(self, strats):
        return strats[list(filter(lambda strat: strats[strat] != None, strats))[0]]


    def allProbability(self, playerOpinions, product): # playerOpinions - мнение игрока player о стратегиях других игроков
        return [playerOpinions[p][product[num]] for num, p in enumerate(playerOpinions.keys())]

    def deleteStratForPlayer(self, player, strat):
        flag = True
        while flag:
            flag = False
            for cell in self.table:
                if cell[player][strat] != None:
                    self.table.remove(cell)
                    flag = True
        self.stratsInitial[player].remove(strat)

    def dominatedStrat(self):
        flag = True
        while flag:
            flag = False
            for player in self.players:
                for strat in self.stratsInitial[player]:
                    gains = [cell[player][strat] for cell in self.modFilter(player, strat, self.table)]
                    for anotherStrat in self.stratsInitial[player]:
                        if strat == anotherStrat: continue
                        anotherGains = [cell[player][anotherStrat] for cell in self.modFilter(player, anotherStrat, self.table)]
                        if all([x >= y for x,y in zip(gains, anotherGains)]) and True in [x > y for x,y in zip(gains, anotherGains)]:
                            self.deleteStratForPlayer(player, anotherStrat)
                            flag = True
                        if flag: break
                    if flag: break
                if flag: break


    def randomGeneric(self):
        return

    def fancyPrint(self, strats):
        if (isinstance(strats, list)):
            for n,l in enumerate(strats):
                print("Оптимальная стратегия #{}".format(n + 1))
                for i in l:
                    print("Игрок - {}, его оптимальная стратегия - {}, выигрыш - {}".format(i, list(l[i].keys())[0], list(l[i].values())[0]))
        else:
            for i in strats:
                print("Игрок - {}, его оптимальная стратегия - {}, выигрыш - {}".format(i, list(strats[i].keys())[0], list(strats[i].values())[0]))
    
    def __str__(self):
        for elem in self.table:
            print(elem)
        return ''


print("Рассмотрим игру трех игроков - мужчина (m), женщина (w) и ребенок (c) с тремя стратегиями - 'пойти на футбол' (f), 'пойти в театр' (t), 'пойти на аттракционы' (a)")
print()

print("Инициализируем таблицу и заполним ее значениями возможных выигрышей игроков")
print()


tBig = Table(('m', 'w', 'c'), {'m': ['f', 't', 'a'], 'w': ['t', 'a'], 'c': ['f', 'a']})
tBig.setAllAtOnce([(4,3,6),  # f t f
                    (2,3,4), # f t a
                    (4,2,6), # f a f
                    (2,5,7), # f a a
                    (3,5,3), # t t f
                    (3,5,4), # t t a
                    (0,2,3), # t a f
                    (0,5,7), # t a a
                    (1,3,3), # a t f
                    (3,3,7), # a t a
                    (4,4,3), # a a f
                    (6,7,10) # a a a
                    ])


print("Расчитаем оптимальную стратегию по критерию Вальда")
tBig.Valdo()
print()

print("Расчитаем оптимальную стратегию по оптимистическому критерию")
tBig.optimistic()
print()

print("Расчитаем оптимальную стратегию по критерию Гурвица")
tBig.Gurwic(0.5)
print()

print("Расчитаем оптимальную стратегию по критерию Лапласа")
tBig.Laplas()
print()

print("Расчитаем оптимальную стратегию по критерию Байеса")
tBig.Bayes({'m': {'w': {'t': 0.8, 'a': 0.2},
                  'c': {'f': 0.5, 'a': 0.5}},
            'w': {'m': {'f': 0.5, 't': 0.3, 'a': 0.2},
                  'c': {'f': 0.5, 'a': 0.5}},
            'c': {'m': {'f': 0.5, 't': 0.2, 'a': 0.3},
                  'w': {'t': 0.6, 'a': 0.4}}})
print()

print("Расчитаем оптимальную стратегию по критерию Сэвиджа")
tBig.Sevij()
print()

print("Расчитаем оптимальную стратегию по критерию Парето")
tBig.Pareto()
print()

print("Расчитаем оптимальную стратегию по критерию Нэша")
tBig.Nesh()
print()

# Выводы и заключение